In [1]:
!pip install pandas

In [2]:
import pandas as pd

#### Separate data string into list where each element is a row

In [3]:
data = 'Airline Code;DelayTimes;FlightCodes;To_From\nAir Canada (!);[21, 40];20015.0;WAterLoo_NEWYork\n<Air France> (12);[];;Montreal_TORONTO\n(Porter Airways. );[60, 22, 87];20035.0;CALgary_Ottawa\n12. Air France;[78, 66];;Ottawa_VANcouvER\n""".\\.Lufthansa.\\.""";[12, 33];20055.0;london_MONTreal\n'

data = data.split('\n')
data

['Airline Code;DelayTimes;FlightCodes;To_From',
 'Air Canada (!);[21, 40];20015.0;WAterLoo_NEWYork',
 '<Air France> (12);[];;Montreal_TORONTO',
 '(Porter Airways. );[60, 22, 87];20035.0;CALgary_Ottawa',
 '12. Air France;[78, 66];;Ottawa_VANcouvER',
 '""".\\.Lufthansa.\\.""";[12, 33];20055.0;london_MONTreal',
 '']

#### Convert each element of the list into its own list separated by the delimiter ';'

In [4]:
data_list = [x.split(';') for x in data]
data_list

[['Airline Code', 'DelayTimes', 'FlightCodes', 'To_From'],
 ['Air Canada (!)', '[21, 40]', '20015.0', 'WAterLoo_NEWYork'],
 ['<Air France> (12)', '[]', '', 'Montreal_TORONTO'],
 ['(Porter Airways. )', '[60, 22, 87]', '20035.0', 'CALgary_Ottawa'],
 ['12. Air France', '[78, 66]', '', 'Ottawa_VANcouvER'],
 ['""".\\.Lufthansa.\\."""', '[12, 33]', '20055.0', 'london_MONTreal'],
 ['']]

#### Remove the last element ['']

In [5]:
data_list = data_list[:-1]
data_list

[['Airline Code', 'DelayTimes', 'FlightCodes', 'To_From'],
 ['Air Canada (!)', '[21, 40]', '20015.0', 'WAterLoo_NEWYork'],
 ['<Air France> (12)', '[]', '', 'Montreal_TORONTO'],
 ['(Porter Airways. )', '[60, 22, 87]', '20035.0', 'CALgary_Ottawa'],
 ['12. Air France', '[78, 66]', '', 'Ottawa_VANcouvER'],
 ['""".\\.Lufthansa.\\."""', '[12, 33]', '20055.0', 'london_MONTreal']]

Convert list of lists into pandas DataFrame

In [6]:
df = pd.DataFrame(data_list[1:], columns=data_list[0])
df

,Airline Code,DelayTimes,FlightCodes,To_From
0,Air Canada (!),"[21, 40]",20015.0,WAterLoo_NEWYork
1,<Air France> (12),[],,Montreal_TORONTO
2,(Porter Airways. ),"[60, 22, 87]",20035.0,CALgary_Ottawa
3,12. Air France,"[78, 66]",,Ottawa_VANcouvER
4,""""""".\.Lufthansa.\.""""""","[12, 33]",20055.0,london_MONTreal


#### Convert data type of flight codes to int and replace null with 0 (for now)

In [7]:
df['FlightCodes'] = pd.to_numeric(df['FlightCodes']).fillna(0).astype(int)
df

,Airline Code,DelayTimes,FlightCodes,To_From
0,Air Canada (!),"[21, 40]",20015,WAterLoo_NEWYork
1,<Air France> (12),[],0,Montreal_TORONTO
2,(Porter Airways. ),"[60, 22, 87]",20035,CALgary_Ottawa
3,12. Air France,"[78, 66]",0,Ottawa_VANcouvER
4,""""""".\.Lufthansa.\.""""""","[12, 33]",20055,london_MONTreal


#### Function to remove punctuation from Airline Codes and apply to first column

In [8]:
def removePunc(s):
    t = ''
    for char in s:
      if char.isalpha() or char == ' ':
        t += char
    return t.strip()
df['Airline Code'] = df['Airline Code'].apply(removePunc)
df

,Airline Code,DelayTimes,FlightCodes,To_From
0,Air Canada,"[21, 40]",20015,WAterLoo_NEWYork
1,Air France,[],0,Montreal_TORONTO
2,Porter Airways,"[60, 22, 87]",20035,CALgary_Ottawa
3,Air France,"[78, 66]",0,Ottawa_VANcouvER
4,Lufthansa,"[12, 33]",20055,london_MONTreal


#### Replace zeros in FlightCodes with previous value + 10

In [9]:
for i, row in df.iterrows():
    if i != 0:
        if row['FlightCodes'] == 0:
            df.at[i,'FlightCodes'] = df.loc[i - 1, 'FlightCodes'] + 10
df

,Airline Code,DelayTimes,FlightCodes,To_From
0,Air Canada,"[21, 40]",20015,WAterLoo_NEWYork
1,Air France,[],20025,Montreal_TORONTO
2,Porter Airways,"[60, 22, 87]",20035,CALgary_Ottawa
3,Air France,"[78, 66]",20045,Ottawa_VANcouvER
4,Lufthansa,"[12, 33]",20055,london_MONTreal


#### Split column 'To_From' by '_'

In [10]:
df[['To', 'From']] = df['To_From'].str.split('_', expand=True)
df = df.drop(['To_From'], axis=1)
print(df)

     Airline Code    DelayTimes  FlightCodes        To       From
0      Air Canada      [21, 40]        20015  WAterLoo    NEWYork
1      Air France            []        20025  Montreal    TORONTO
2  Porter Airways  [60, 22, 87]        20035   CALgary     Ottawa
3      Air France      [78, 66]        20045    Ottawa  VANcouvER
4       Lufthansa      [12, 33]        20055    london   MONTreal


#### Define capital case function and apply to columns 'To' and 'From' to convert them to capital case

In [11]:
def capitalCase(s):
  return s[0].upper() + s[1:].lower()

df['To'] = df['To'].apply(capitalCase)
df['From'] = df['From'].apply(capitalCase)
df

,Airline Code,DelayTimes,FlightCodes,To,From
0,Air Canada,"[21, 40]",20015,Waterloo,Newyork
1,Air France,[],20025,Montreal,Toronto
2,Porter Airways,"[60, 22, 87]",20035,Calgary,Ottawa
3,Air France,"[78, 66]",20045,Ottawa,Vancouver
4,Lufthansa,"[12, 33]",20055,London,Montreal


#### Improve readibility of column names (add space to DelayTimes and FlightCodes)

In [12]:
df = df.rename(columns={'DelayTimes': 'Delay Times', 'FlightCodes': 'Flight Codes'})
df

,Airline Code,Delay Times,Flight Codes,To,From
0,Air Canada,"[21, 40]",20015,Waterloo,Newyork
1,Air France,[],20025,Montreal,Toronto
2,Porter Airways,"[60, 22, 87]",20035,Calgary,Ottawa
3,Air France,"[78, 66]",20045,Ottawa,Vancouver
4,Lufthansa,"[12, 33]",20055,London,Montreal


#### Change 'Newyork' to 'New York'

In [17]:
df.at[0, 'From'] = 'New York'
df

,Airline Code,Delay Times,Flight Codes,To,From
0,Air Canada,"[21, 40]",20015,Waterloo,New York
1,Air France,[],20025,Montreal,Toronto
2,Porter Airways,"[60, 22, 87]",20035,Calgary,Ottawa
3,Air France,"[78, 66]",20045,Ottawa,Vancouver
4,Lufthansa,"[12, 33]",20055,London,Montreal
